Pipeline A: Whisper -> LLM-Diarisierung
Kein Chunking, kein JSON
Output: [Sprecher 1]: ... 

In [1]:
### Imports + Konfiguration
import os
from pathlib import Path
import requests
from dotenv import load_dotenv

load_dotenv()

VLLM_BASE_URL = os.getenv("VLLM_BASE_URL", "http://127.0.0.1:8005/v1")
VLLM_MODEL = os.getenv("VLLM_MODEL", "openai/gpt-oss-120b")

# Repository-Root: individuell anpassen
REPO_ROOT = Path("~/jupyter/diarization-benchmark").expanduser().resolve()

# Pfad zu gespeicherten Audios
AUDIO_DIR = REPO_ROOT / "data" / "audio"

# Test-Audio-Datei (kann später weg), Endung beliebig -> normalisieren es später
TEST_AUDIO = REPO_ROOT /"data" / "test_audio" / "test1.mp3"

# Path-Debugging
print("REPO_ROOT:", REPO_ROOT)
print("TEST_AUDIO exists:", TEST_AUDIO.exists(), TEST_AUDIO)


REPO_ROOT: /home/liegepa/jupyter/diarization-benchmark
TEST_AUDIO exists: True /home/liegepa/jupyter/diarization-benchmark/data/test_audio/test1.mp3


In [5]:
### vLLM-client, OpenAI-kompatibel
import requests

def chat_vllm(messages, model=VLLM_MODEL, temperature=0.0, max_tokens=800):
    url = f"{VLLM_BASE_URL}/chat/completions"
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    data = requests.post(url, json=payload, timeout=600).json()
    return data["choices"][0]["message"]["content"]

# Erster Test
print(chat_vllm([{"role":"user","content":"Antworte nur mit Ja."}]))

Ja


Whisper-Transkription der Audio via faster-whisper
- lokal reproduzierbar, keine Cloud
- wir nehmen Segment-Zeitstempel für späteren Pyannote-Merge (nicht benötigt für LLM-Diarisierung)

In [3]:
### Code für Transkription
from faster_whisper import WhisperModel

WHISPER_MODEL_SIZE = os.getenv("WHISPER_MODEL_SIZE", "small") # Kleines Modell für Tests
# WHISPER_MODEL_SIZE = os.getenv("WHISPER_MODEL_SIZE", "large-v3") # Großes Modell für Prod
WHISPER_DEVICE = os.getenv("WHISPER_DEVICE", "cpu")  # Für unseren Test auf CPU laufen lassen -> langsamer
# WHISPER_DEVICE = os.getenv("WHISPER_DEVICE", "cuda") # Wenn verfügbar: Auf GPU laufen lassen -> schneller -> Aber: Konfig-Anpassungen notwendig!
WHISPER_COMPUTE_TYPE = os.getenv("WHISPER_COMPUTE_TYPE", "int8")  # cpu: int8 gut

_whisper_model = None

def get_whisper_model():
    global _whisper_model
    if _whisper_model is None:
        _whisper_model = WhisperModel(
            WHISPER_MODEL_SIZE,
            device=WHISPER_DEVICE,
            compute_type=WHISPER_COMPUTE_TYPE,
        )
    return _whisper_model

def transcribe_faster_whisper(audio_path: str, language: str | None = None):
    """
    Returns:
      transcript: str (full text)
      segments: list of dicts: [{"start": float, "end": float, "text": str}, ...]
    """
    model = get_whisper_model()
    segments_iter, info = model.transcribe(
        audio_path,
        language=language,
        vad_filter=True,
        word_timestamps=False,  # später ggf. True, falls wir word-level brauchen
    )
    segments = []
    texts = []
    for seg in segments_iter:
        txt = seg.text.strip()
        segments.append({"start": float(seg.start), "end": float(seg.end), "text": txt})
        texts.append(txt)
    transcript = "\n".join(texts).strip()
    return transcript, segments, info

# Quick test, wenn TEST_AUDIO gesetzt ist
if TEST_AUDIO:
    t, segs, info = transcribe_faster_whisper(TEST_AUDIO, language="de")
    print("Language:", info.language, "Prob:", info.language_probability)
    print("Transcript:\n", t[:1000])
    print("\nFirst segments:", segs[:3])


Language: de Prob: 1
Transcript:
 Guten Tag, Krilloffice, mein Name ist Ernst Harz, sagen Sie mir bitte, heißen Sie?
Ich heiße Brinkmann Maria.
Frau Brinkmann, könnten Sie bitte Ihren Namen langsam Buch stabilieren?
B-R-I-N-K-M-A-Doppel-N.
Okay, und der Vorname?
Maria M-A-R-I-A.
Gut, Frau Brinkmann, wann sind Sie geboren und wie alt sind Sie?
Ich bin am 15.15.1972 geboren und das heißt, ich bin 47 Jahre alt.
Prima.
Frau Brinkmann, sagen Sie bitte, was für eine Sitzung ist?
Ja, Herr Doktor, ich habe so einen Schmerzen hier im Handgelenk und in meinen Sprunggelenk.
Ich war auf dem Weg zur Arbeit mit meinem Fahrrad heute Morgen
und dann war mir ein bisschen schwindelig
und dann bin ich mit dem Fahrrad gestürzt.
Und ich bin so auf meiner, wie heißt das?
Ich bin auf meiner rechten Hand, so mit ausgestrecktem Hand, so auf dem Boden gefallen
und mein Sprunggelenk ist auch umgeknippt während des Umfalls.
Gut, also Frau Brinkmann, Sie sagen, Sie haben jetzt momentan Schmerzen,
brauchen Sie jetz

### LLM-Diarisierung Prompt
System- und User-Prompts

In [4]:
def diarize_with_llm(transcript: str):
    system_prompt = (
        "You are a professional conversation diarization engine. "
        "Assign speaker labels logically solely based on text. "
        "Do not add, remove, or summarize content. "
        "Make sure before assignment that the context fits (a doctor will not say things a patient would say for example). "
        "Output only the diarized transcript."
    )

    user_prompt = (
        "Add speaker labels to the following transcript using logic to determine which sentence belongs to which speaker. "
        "For assigning sentences: Make out the role of the speaker (for example doctor). Keep this in mind when assigning speaker names. "
        "Use '[Sprecher 1]', '[Sprecher 2]', ... for the different speakers. "
        "Check thoroughly for every sentence if the assignment to the speaker is contextually and logically plausible. "
        "For example, in a doctor-patient-dialogue, do not assign medical questions to the Speaker who was identified as a patient before. "
        "Produce a readable dialogue format between Speakers.\n\n"
        f"{transcript}"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    return chat_vllm(messages, temperature=0.0, max_tokens=2000)

if TEST_AUDIO:
    transcript, segs, info = transcribe_faster_whisper(TEST_AUDIO, language="de")
    diarized = diarize_with_llm(transcript)
    print(diarized)


None
